## Install required libraries

- `azure-identity`: Azure Active Directory identity client library
- `azure.mgmt.confidentialledger`: Azure confidential ledger control plane client library
- `azure.confidentialledger`:  Azure confidential ledger data plane client library

In [ ]:
!pip install azure-identity azure.mgmt.confidentialledger azure.confidentialledger 

## Initialize

Import the required libraries.

In [ ]:
# Import the Azure authentication library

from azure.identity import DefaultAzureCredential, ManagedIdentityCredential, AzureCliCredential, DeviceCodeCredential

## Import the control plane sdk

from azure.mgmt.confidentialledger import ConfidentialLedger as ConfidentialLedgerAPI
from azure.mgmt.confidentialledger.models import ConfidentialLedger

# import the data plane sdk

from azure.confidentialledger import ConfidentialLedgerClient
from azure.confidentialledger.certificate import ConfidentialLedgerCertificateClient

We'll finish setup by setting some variables for use in your application: the resource group, the name of ledger you want to create, and two urls to be used by the data plane client library.

In [ ]:
resource_group = "azureailab31"
ledger_name = "azureailab31"
subscription_id = "82ed3e67-3850-491b-a098-cdca6aa71f3f"

identity_url = "https://identity.confidential-ledger.core.azure.com"
ledger_url = "https://" + ledger_name + ".confidential-ledger.azure.com"

In [ ]:
credential = DefaultAzureCredential()

## Use the control plane library

The control plane client library (azure.mgmt.confidentialledger) allows operations on ledgers, such as creation, modification, and deletion, listing the ledgers associated with a subscription, and getting the details of a specific ledger.

In our code, we will first create a control plane client by passing the ConfidentialLedgerAPI the credential variable and your Azure subscription ID (both of which are set above).

In [ ]:
confidential_ledger_mgmt = ConfidentialLedgerAPI(
    credential, subscription_id
)

We can now create a ledger using `begin_create`. The `begin_create` function requires three parameters: your resource group, a name for the ledger, and a "properties" object.

Create a properties dictionary with the following keys and values, and assign it to a variable.

In [ ]:
properties = {
    "location": "eastus",
    "tags": {},
    "properties": {
        "ledgerType": "Public",
        "aadBasedSecurityPrincipals": [],
    },
}

ledger_properties = ConfidentialLedger(**properties)

Now pass the resource group, the name of your ledger, and the properties object to `begin_create`.

In [ ]:
confidential_ledger_mgmt.ledger.begin_create(resource_group, ledger_name, ledger_properties)

To verify that your ledger was successfully created, view its details using the `get` function.

In [ ]:
myledger = confidential_ledger_mgmt.ledger.get(resource_group, ledger_name)

print("Here are the details of your newly created ledger:")
print (f"- Name: {myledger.name}")
print (f"- Location: {myledger.location}")
print (f"- ID: {myledger.id}")

Check some additional properties of the ledger.

In [ ]:
print(myledger.properties)

### Exercise:

Open the Azure portal, locate the Azure item of type `Confidential Ledger` and explore its properties.

Refresh periodically the `Overview` section until the `Identity Service Endpoint` and `Ledger Endpoint` values are populated (i.e., they are different from `--`).

>**NOTE**:
>
>You need to wait until these values are populated, which means the Confidential Ledger instance has been properly initialized. Advancing in the notebook before this process is complete will most likely result in errors being thrown by subsequent cells.


## Use the data plane client library

Now that we have a ledger, we'll interact with it using the data plane client library (azure.confidentialledger).

First, we will generate and save a confidential ledger certificate.

In [ ]:
identity_client = ConfidentialLedgerCertificateClient(identity_url)
network_identity = identity_client.get_ledger_identity(
     ledger_id=ledger_name
)

ledger_tls_cert_file_name = "networkcert.pem"
with open(ledger_tls_cert_file_name, "w") as cert_file:
    cert_file.write(network_identity['ledgerTlsCertificate'])

Now we can use the network certificate, along with the ledger URL and our credentials, to create a confidential ledger client.

In [ ]:
credential = DeviceCodeCredential()

ledger_client = ConfidentialLedgerClient(
     endpoint=ledger_url, 
     credential=credential,
     ledger_certificate_path=ledger_tls_cert_file_name
)

We are prepared to write to the ledger. We will do so using the `create_ledger_entry` function.

The print function will return the transaction ID of your write to the ledger, which can be used to retrieve the message you wrote to the ledger.

In [ ]:
sample_entry = {"contents": "Hellow immutable, blockchain-based world!"}
append_result = ledger_client.create_ledger_entry(entry=sample_entry)
print(f'Transaction id: {append_result["transactionId"]}')

Get the details of the ledger transaction based on the transaction id.

In [ ]:
entry = ledger_client.get_ledger_entry(transaction_id=append_result['transactionId'])['entry']
print(f"Entry (transaction id = {entry['transactionId']}) in collection {entry['collectionId']}: {entry['contents']}")

Get the details of the last transaction from the ledger.

In [ ]:
latest_entry = ledger_client.get_current_ledger_entry()
print(f"Current entry (transaction id = {latest_entry['transactionId']}) in collection {latest_entry['collectionId']}: {latest_entry['contents']}")

## Using the polling pattern

If you'd like to wait for your write transaction to be committed to your ledger, you can use the `begin_create_ledger_entry` function. This will return a poller to wait until the entry is durably committed.

In [ ]:
sample_entry = {"contents": "Hellow immutable, blockchain-based world! Again!"}
ledger_entry_poller = ledger_client.begin_create_ledger_entry( 
    entry=sample_entry
)
ledger_entry_result = ledger_entry_poller.result()
ledger_entry_result

## Analyzing transaction receipts

Get the transaction receipt based on the transaction id.

In [ ]:
get_receipt_poller = ledger_client.begin_get_receipt(ledger_entry_result["transactionId"])
get_receipt_result = get_receipt_poller.result() 
get_receipt_result

The JSON response contains the following fields at the root level.

- **receipt**: It contains the values that can be used to verify the validity of the receipt for the corresponding write transaction.
- **state**: The status of the returned JSON response. The following are the possible values allowed:
    - Ready: The receipt returned in the response is available
    - Loading: The receipt isn't yet available to be retrieved and the request will have to be retried
- **transactionId**: The transaction ID associated with the write transaction receipt.

The receipt field contains the following fields.

- **cert**: String with the [PEM](https://en.wikipedia.org/wiki/Privacy-Enhanced_Mail) public key certificate of the Confidential Consortium Framework (CCF) node that signed the write transaction. The certificate of the signing node should always be endorsed by the service identity certificate. See also more details about how transactions get regularly signed and how the signature transactions are appended to the ledger in CCF at the following [link](https://microsoft.github.io/CCF/main/architecture/merkle_tree.html).

- **is_signature_transaction**: Boolean value indicating whether the receipt is related to a signature transaction or not. Receipts for signature transactions can't be retrieved for Confidential Ledgers.

- **node_id**: Hexadecimal string representing the [SHA-256](https://en.wikipedia.org/wiki/SHA-2) hash digest of the public key of the signing CCF node.

- **leaf_components**: The components of the leaf node hash in the [Merkle Tree](https://en.wikipedia.org/wiki/Merkle_tree) that are associated to the specified transaction. A Merkle Tree is a tree data structure that records the hash of every transaction and guarantees the integrity of the ledger. For more information on how a Merkle Tree is used in CCF, see the related [CCF documentation](https://microsoft.github.io/CCF/main/architecture/merkle_tree.html).

- **proof**: List of key-value pairs representing the Merkle Tree nodes hashes that, when combined with the leaf node hash corresponding to the given transaction, allow the recomputation of the root hash of the tree. Thanks to the properties of a Merkle Tree, it's possible to recompute the root hash of the tree only a subset of nodes. The elements in this list are in the form of key-value pairs: keys indicate the relative position with respect to the parent node in the tree at a certain level; values are the SHA-256 hash digests of the node given, as hexadecimal strings.

- **service_endorsements**: List of PEM-encoded certificates strings representing previous service identities certificates. It's possible that the service identity that endorsed the signing node isn't the same as the one that issued the receipt. For example, the service certificate is renewed after a disaster recovery of a Confidential Ledger. The list of past service certificates allows auditors to build the chain of trust from the CCF signing node to the current service certificate.

- **signature**: Base64 string representing the signature of the root of the Merkle Tree at the given transaction, by the signing CCF node.

The leaf_components field contains the following fields.

- **claims_digest**: Hexadecimal string representing the SHA-256 hash digest of the [application claim](https://microsoft.github.io/CCF/main/use_apps/verify_tx.html#application-claims) attached by the Confidential Ledger application at the time the transaction was executed. Application claims are currently unsupported as the Confidential Ledger application doesn't attach any claim when executing a write transaction.

- **commit_evidence**: A unique string produced per transaction, derived from the transaction ID and the ledger secrets. For more information about the commit evidence, see the related [CCF documentation](https://microsoft.github.io/CCF/main/use_apps/verify_tx.html#commit-evidence).

- **write_set_digest**: Hexadecimal string representing the SHA-256 hash digest of the [Key-Value store](https://microsoft.github.io/CCF/main/build_apps/kv/index.html), which contains all the keys and values written at the time the transaction was completed. For more information about the write set, see the related [CCF documentation](https://microsoft.github.io/CCF/main/use_apps/verify_tx.html#commit-evidence).